# A simple DNN model built in Keras.

Let's start off with the Python imports that we need.

In [1]:
import os, json, math
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0-alpha0


## Locating the CSV files

We will start with the CSV files that we wrote out in the [first notebook](../01_explore/taxifare.iypnb) of this sequence. Just so you don't have to run the notebook, we saved a copy in ../data

In [2]:
!ls -l ../data/*.csv

-rw-r--r-- 1 jupyter jupyter  85978 Jun  1 18:03 ../data/taxi-test.csv
-rw-r--r-- 1 jupyter jupyter 402560 Jun  1 18:03 ../data/taxi-train.csv
-rw-r--r-- 1 jupyter jupyter  85797 Jun  1 18:03 ../data/taxi-valid.csv


## Use tf.data to read the CSV files

We wrote these cells in the [third notebook](../03_tfdata/input_pipeline.ipynb) of this sequence.

In [3]:
CSV_COLUMNS  = ['fare_amount', 
                'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 
                'passenger_count', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS     = [[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],['na']]

In [4]:
def features_and_labels(row_data):
    for unwanted_col in ['key']:
        row_data.pop(unwanted_col)
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label  # features, label

# load the training data
def load_dataset(pattern, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
  dataset = (tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS)
             .map(features_and_labels) # features, label
             .cache())
  if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(1000).repeat()
  dataset = dataset.prefetch(1) # take advantage of multi-threading; 1=AUTOTUNE
  return dataset

In [22]:
## Build a simple Keras DNN using its Functional API
def transform(inputs):
    preproc = inputs.copy()
    # scale the lat, lon values to be in -1, 1
    for lon_col in ['pickup_longitude', 'dropoff_longitude']:
        preproc[lon_col] = tf.keras.layers.Lambda(lambda x: x/180.0, name='scale_{}'.format(lon_col))(inputs[lon_col])
    for lat_col in ['pickup_latitude', 'dropoff_latitude']:
        preproc[lat_col] = tf.keras.layers.Lambda(lambda x: x/90.0, name='scale_{}'.format(lat_col))(inputs[lat_col])
    return preproc

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true))) 

def build_dnn_model():
    INPUT_COLS = ['pickup_longitude', 'pickup_latitude', 
                  'dropoff_longitude', 'dropoff_latitude', 
                  'passenger_count']

    # input layer
    inputs = {
        colname : tf.keras.layers.Input(name=colname, shape=(), dtype='float32')
           for colname in INPUT_COLS
    }
    
    preproc = transform(inputs)
    
    # the DenseFeatures layer takes all the inputs and creates a single input layer
    feature_columns = [
        tf.feature_column.numeric_column(colname)
           for colname in INPUT_COLS
    ]
    dnn_inputs = tf.keras.layers.DenseFeatures(feature_columns)(preproc)

    # two hidden layers of [64, 8] just in like the BQML DNN
    h1 = tf.keras.layers.Dense(64, activation='relu')(dnn_inputs)
    h2 = tf.keras.layers.Dense(8, activation='relu')(h1)
    # final output would normally have a linear activation because this is regression
    # However, we know something about the taxi problem (fares are between $2.5 and below $60).
    # Use that here. (You can verify by running this query):
    # SELECT APPROX_QUANTILES(fare_amount, 100) FROM serverlessml.cleaned_training_data
    fare_thresh = lambda x: tf.keras.activations.relu(x, max_value=60, threshold=2.5)
    output = tf.keras.layers.Dense(1, activation=fare_thresh)(h2)
    model = tf.keras.models.Model(inputs, output)
    model.compile(optimizer='adam', loss='mse', metrics=[rmse, 'mse'])
    return model

model = build_dnn_model()
print(model.summary())

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dropoff_latitude (InputLayer)   [(None,)]            0                                            
__________________________________________________________________________________________________
dropoff_longitude (InputLayer)  [(None,)]            0                                            
__________________________________________________________________________________________________
pickup_latitude (InputLayer)    [(None,)]            0                                            
__________________________________________________________________________________________________
pickup_longitude (InputLayer)   [(None,)]            0                                            
____________________________________________________________________________________________

## Train model

To train the model, call model.fit()

In [ ]:
TRAIN_BATCH_SIZE = 32
NUM_TRAIN_EXAMPLES = 10000 * 5 # remember the training dataset repeats, so we can specify anything here
NUM_EVALS = 5  # evaluate this many times

trainds = load_dataset('../data/taxi-train*', TRAIN_BATCH_SIZE, tf.estimator.ModeKeys.TRAIN)
evalds = load_dataset('../data/taxi-valid*', 1024, tf.estimator.ModeKeys.EVAL)

steps_per_epoch = NUM_TRAIN_EXAMPLES // (TRAIN_BATCH_SIZE * NUM_EVALS)
history = model.fit(trainds, 
                    validation_data=evalds,
                    epochs=NUM_EVALS, 
                    steps_per_epoch=steps_per_epoch)

Epoch 1/5
310/312 [============================>.] - ETA: 0s - loss: 221.6176 - rmse: 14.5495 - mse: 221.6176

In the next notebook, we will improve this model through feature engineering.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.